In [9]:
import os
import getpass


from utils import *
from keras.models import Sequential, Model, load_model
from keras.layers import Flatten, Dense, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pickle as cPickle
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [10]:
from matplotlib import pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score,confusion_matrix,multilabel_confusion_matrix,recall_score
from sklearn.preprocessing import normalize
from sklearn import svm
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [11]:
fname_model = 'patch_based_2048_512_128_8.h5'

In [12]:
#user defined variables
version='imgsize64_batchsize64_4layers_2048_512_128'
IMG_SIZE   = 32 #deafult 32
BATCH_SIZE  = 64 #default 16
DATASET_DIR = '../../../Databases/MIT_split'#'/home/mcv/datasets/MIT_split'
MODEL_FNAME = 'output_results/'+fname_model

# if not os.path.exists(DATASET_DIR):
#   print(Color.RED, 'ERROR: dataset directory '+DATASET_DIR+' does not exist!\n')
#   quit()


print('Building MLP model...\n')

#Build the Multi Layer Perceptron model

####
# Split into tiles
####

model = Sequential()
model.add(Reshape((IMG_SIZE*IMG_SIZE*3,),input_shape=(IMG_SIZE, IMG_SIZE, 3),name='first'))
model.add(Dense(units=2048, activation='relu',name='second'))
model.add(Dense(units=512, activation='relu', name='3'))
model.add(Dense(units=128, activation='relu', name='4'))
model.add(Dense(units=8, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

print(model.summary())
#plot_model(model, to_file=f'output_results/feature_extraction_{version}.png', show_shapes=True, show_layer_names=True)

model.load_weights(MODEL_FNAME)
#plot_model(model, to_file=f'output_results/feature_extraction_{version}.png', show_shapes=True, show_layer_names=True)

print('Done!\n')

Building MLP model...

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
first (Reshape)              (None, 3072)              0         
_________________________________________________________________
second (Dense)               (None, 2048)              6293504   
_________________________________________________________________
3 (Dense)                    (None, 512)               1049088   
_________________________________________________________________
4 (Dense)                    (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 1032      
Total params: 7,409,288
Trainable params: 7,409,288
Non-trainable params: 0
_________________________________________________________________
None
Done!



In [13]:
model_layer = Model(inputs=model.input, outputs=model.get_layer('second').output)

In [14]:
'''
def load_images_from_folder(folder):
    images = []
    for class_folder in os.listdir(folder):
        for filename in os.listdir(folder+'/'+class_folder):
            #print()
            img = Image.open(os.path.join(folder+'/'+class_folder,filename))
            if img is not None:
                
                images.append(img)
            Image.close(os.path.join(folder+'/'+class_folder,filename))


    return images
'''
def load_images_from_folder(folder):
    images=[]
    for path, _, files in os.walk(folder):
        for file in files:
            
            img = Image.open(path+'/'+file) 
            if img is not None:
                
                images.append(img.copy())
            img.close()
    return images
        

In [15]:
train_images_folder = '../mcv/datasets/Patches_P32_B64_E50/train'
test_images_folder = '../mcv/datasets/Patches_P32_B64_E50/test'
train_labels = cPickle.load(open('../mcv/datasets/train_labels.dat','rb'))
test_labels = cPickle.load(open('../mcv/datasets/test_labels.dat','rb'))

In [16]:
train_images = load_images_from_folder(train_images_folder)
test_images = load_images_from_folder(test_images_folder)

In [17]:
n_patches = int(((256)/32)**2)
print(n_patches)

64


In [18]:
## TRAIN ##
patch_idx=0
Train_descriptors=[]
Train_label_per_descriptor=[]
for labels in train_labels:
    patches=[]
    Train_descriptors_patch=[]
    for i in range(n_patches):
        patches.append(np.asarray(train_images[i+patch_idx]))
        
    for patch in patches:    
        patch = np.expand_dims(np.resize(patch, (IMG_SIZE, IMG_SIZE, 3)), axis=0)
        Train_descriptors_patch.append(model_layer.predict(patch/255.0))
        
    Train_label_per_descriptor.append(labels)
        
    Train_descriptors.append(Train_descriptors_patch)
    Train_label_per_descriptor.append(labels)
    patch_idx+=n_patches

In [19]:
with open('train_descriptors_P32_2048.pkl', 'wb') as f:
    cPickle.dump(Train_descriptors, f)

In [20]:
## TEST ##
patch_idx=0
Test_descriptors=[]
Test_label_per_descriptor=[]


for labels in test_labels:
    patches=[]
    Test_descriptors_patch=[]
    for i in range(n_patches):
        patches.append(np.asarray(test_images[i+patch_idx]))
        
    for patch in patches:    
        patch = np.expand_dims(np.resize(patch, (IMG_SIZE, IMG_SIZE, 3)), axis=0)
        Test_descriptors_patch.append(model_layer.predict(patch/255.0))
        
    Test_label_per_descriptor.append(labels)
        
    Test_descriptors.append(Test_descriptors_patch)
    Test_label_per_descriptor.append(labels)
    patch_idx+=n_patches
    
with open('test_descriptors_P32_2048.pkl', 'wb') as f:
    cPickle.dump(Test_descriptors, f)

### Linear kernel only SVM without codebook

In [ ]:
Train_descriptors=[i[0] for i in Train_descriptors]

In [ ]:
Test_descriptors=[i[0] for i in Test_descriptors]

In [ ]:
lin_clf = svm.LinearSVC(max_iter=2000, C=10)
lin_clf.fit(Train_descriptors, train_labels)

scores = cross_val_score(lin_clf, Test_descriptors, Test_label_per_descriptor, cv=5)
accuracy=scores.mean()*100

print(accuracy)

### RBF Kernel only SVM without codebook

In [ ]:
rbf_svc = svm.SVC(kernel='rbf', C=1000, gamma=0.001)
rbf_svc.fit(Train_descriptors, train_labels)

scores = cross_val_score(rbf_svc, Test_descriptors, test_labels, cv=5)

accuracy=scores.mean()*100

print(accuracy)

### BoW with Linear Kernel

In [22]:
train_descriptors_bow = [des for descriptors_patches in Train_descriptors for des in descriptors_patches]

In [23]:
train_descriptors_bow = [des for descriptors_patches in Train_descriptors for des in descriptors_patches]
D=np.vstack(train_descriptors_bow)

k=128
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

MiniBatchKMeans(batch_size=2560, compute_labels=False, init='k-means++',
                init_size=None, max_iter=100, max_no_improvement=10,
                n_clusters=128, n_init=3, random_state=42,
                reassignment_ratio=0.0001, tol=0.0, verbose=False)

In [24]:
visual_words=np.zeros((len(Train_descriptors),k*n_patches),dtype=np.float32)
for idx,patches in enumerate(Train_descriptors):

    image_histograms = []
    for id2,image_patch in enumerate(patches):
        patch_words = codebook.predict(image_patch)
        image_histograms.append(normalize(np.bincount(patch_words,minlength=k).reshape(1,-1)))
        # concatenated histogram has k(1+2**(2pyramid_levels)) bins
        # normalize(np.bincount(words,minlength=k).reshape(1,-1))

    concatenated_histogram=image_histograms[0].copy()
    for patch_idx in range(1,len(image_histograms)):
        # concatenate histograms
        concatenated_histogram = np.concatenate((concatenated_histogram,image_histograms[patch_idx]))
    visual_words[idx,:]=concatenated_histogram.flatten()

In [25]:
scaler = StandardScaler()
scaler.fit_transform(visual_words)

array([[-0.12292531, -0.10625593, -0.04616341, ..., -0.02306328,
        -0.09549959, -0.06111731],
       [-0.12292531, -0.10625593, -0.04616341, ..., -0.02306328,
        -0.09549959, -0.06111731],
       [-0.12292531, -0.10625593, -0.04616341, ..., -0.02306328,
        -0.09549959, -0.06111731],
       ...,
       [-0.12292531, -0.10625593, -0.04616341, ..., -0.02306328,
        -0.09549959, -0.06111731],
       [-0.12292531, -0.10625593, -0.04616341, ..., -0.02306328,
        -0.09549959, -0.06111731],
       [-0.12292531, -0.10625593, -0.04616341, ..., -0.02306328,
        -0.09549959, -0.06111731]], dtype=float32)

In [26]:
len(Test_descriptors[0])

64

In [27]:
lin_clf = svm.LinearSVC(max_iter=2000, C=10)
lin_clf.fit(visual_words, train_labels)

visual_words_test=np.zeros((len(Test_descriptors),k*n_patches),dtype=np.float32)
for idx,patches in enumerate(Test_descriptors):

    image_histograms = []
    for id2,image_patch in enumerate(patches):
        patch_words = codebook.predict(image_patch)
        image_histograms.append(normalize(np.bincount(patch_words,minlength=k).reshape(1,-1)))
        # concatenated histogram has k(1+2**(2pyramid_levels)) bins
        # normalize(np.bincount(words,minlength=k).reshape(1,-1))

    concatenated_histogram=image_histograms[0].copy()
    for patch_idx in range(1,len(image_histograms)):
        # concatenate histograms
        concatenated_histogram = np.concatenate((concatenated_histogram,image_histograms[patch_idx]))
    visual_words_test[idx,:]=concatenated_histogram.flatten()

visual_words_test=scaler.transform(visual_words_test)
scores = cross_val_score(lin_clf, visual_words_test, test_labels, cv=5)
accuracy=scores.mean()*100

print(accuracy)

46.951154052603336


### BoW with RBF Kernel

In [ ]:
train_descriptors_bow = [des for descriptors_patches in Train_descriptors for des in descriptors_patches]
D=np.vstack(train_descriptors_bow)

k=128
codebook = MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4,random_state=42)
codebook.fit(D)

visual_words=np.zeros((len(Train_descriptors),k),dtype=np.float32)
for i in range(len(Train_descriptors)):
    words=codebook.predict(Train_descriptors[i])
    visual_words[i,:]=normalize(np.bincount(words,minlength=k).reshape(1,-1))

# knn = KNeighborsClassifier(n_neighbors=37,n_jobs=-1,metric='manhattan')
# knn.fit(visual_words, train_labels)

today = datetime.now()
dt_string = today.strftime("%H:%M:%S")
print(f"{dt_string} started doing")


In [ ]:
scaler = StandardScaler()
scaler.fit_transform(visual_words)

In [28]:
rbf_svc = svm.SVC(kernel='rbf', C=1000, gamma=0.001)
rbf_svc.fit(visual_words, train_labels)

visual_words_test=np.zeros((len(Test_descriptors),k*n_patches),dtype=np.float32)
for idx,patches in enumerate(Test_descriptors):

    image_histograms = []
    for id2,image_patch in enumerate(patches):
        patch_words = codebook.predict(image_patch)
        image_histograms.append(normalize(np.bincount(patch_words,minlength=k).reshape(1,-1)))
        # concatenated histogram has k(1+2**(2pyramid_levels)) bins
        # normalize(np.bincount(words,minlength=k).reshape(1,-1))

    concatenated_histogram=image_histograms[0].copy()
    for patch_idx in range(1,len(image_histograms)):
        # concatenate histograms
        concatenated_histogram = np.concatenate((concatenated_histogram,image_histograms[patch_idx]))
    visual_words_test[idx,:]=concatenated_histogram.flatten()

visual_words_test=scaler.transform(visual_words_test)
scores = cross_val_score(rbf_svc, visual_words_test, test_labels, cv=5)
accuracy=scores.mean()*100

print(accuracy)

14.621578099838967
